This case study examins the use of patsy, statsmodels and scikit-learn to do classical statistical models, and to make use of basic machine learning for predicting and classifying data.

In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame({
    'x0': [1,2,3,4,5],
    'x1': [0.01, -0.01, 0.25, -4.1, 0.0],
    'y':  [-1.5, 0, 3.6, 1.3, -2]
})

data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [2]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [3]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [5]:
model_cols= ['x0' , 'x1']
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

y ~ x0 + x1

In [6]:
import patsy

In [8]:
y,X = patsy.dmatrices('y ~ x0 + x1',data)

In [9]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [10]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

In [11]:
coef, reid, _, _, = np.linalg.lstsq(X, y)

<ipython-input-11-339310a0f762>:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  coef, reid, _, _, = np.linalg.lstsq(X, y)


In [12]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [13]:
coef = pd.Series(coef.squeeze(), index=X.design_info.column_names)

In [14]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

In [15]:
y, X = patsy.dmatrices('y ~ np.log(np.abs(x1)+1)', data)

In [16]:
X

DesignMatrix with shape (5, 2)
  Intercept  np.log(np.abs(x1) + 1)
          1                 0.00995
          1                 0.00995
          1                 0.22314
          1                 1.62924
          1                 0.00000
  Terms:
    'Intercept' (column 0)
    'np.log(np.abs(x1) + 1)' (column 1)

In [17]:
y, X = patsy.dmatrices(' y ~ standardize(x0) + center(x1)', data)

In [18]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

In [19]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [20]:
new_data = pd.DataFrame({
    'x0' : [6,7,8,9],
    'x1' : [3.1,-0.5,0.0,2.3],
    'y' : [1,2,3,4],
})

new_data

,x0,x1,y
0,6,3.1,1
1,7,-0.5,2
2,8,0.0,3
3,9,2.3,4


In [21]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [23]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

In [24]:
y, X = patsy.dmatrices('y ~ I(x0 + x1)', data)

In [25]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

In [26]:
dir(patsy.builtins)

['C',
 'ContrastMatrix',
 'Diff',
 'Helmert',
 'I',
 'Poly',
 'Q',
 'Sum',
 'Treatment',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'bs',
 'cc',
 'center',
 'cr',
 'scale',
 'standardize',
 'te',
 'test_I',
 'test_Q']

In [27]:
print(patsy.builtins.Q.__doc__)

A way to 'quote' variable names, especially ones that do not otherwise
    meet Python's variable name rules.

    If ``x`` is a variable, ``Q("x")`` returns the value of ``x``. (Note that
    ``Q`` takes the *string* ``"x"``, not the value of ``x`` itself.) This
    works even if instead of ``x``, we have a variable name that would not
    otherwise be legal in Python.

    For example, if you have a column of data named ``weight.in.kg``, then you
    can't write::

      y ~ weight.in.kg

    because Python will try to find a variable named ``weight``, that has an
    attribute named ``in``, that has an attribute named ``kg``. (And worse
    yet, ``in`` is a reserved word, which makes this example doubly broken.)
    Instead, write::

      y ~ Q("weight.in.kg")

    and all will be well. Note, though, that this requires embedding a Python
    string inside your formula, which may require some care with your quote
    marks. Some standard options include::

      my_fit_function("y ~

In [28]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [29]:
np.random.seed(12345)

In [30]:
def dnorm(mean, variance, size=1):
    if isinstance(size, int):
        size = (size,)
    return mean + np.sqrt(variance) * np.random.randn(*size)

N = 100
X = np.c_[dnorm(0, 0.4, size=N),
           dnorm(0, 0.6, size=N),
           dnorm(0, 0.2, size=N)]

In [31]:
eps = dnorm(0, 0.1, size=N)
beta = [0.1, 0.3, 0.5]

y = np.dot(X, beta) + eps

In [33]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [34]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

In [35]:
X_model = sm.add_constant(X)
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

In [36]:
model = sm.OLS(y, X)

In [37]:
result = model.fit()
result.params

array([0.17826108, 0.22303962, 0.50095093])

In [38]:
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Tue, 16 Nov 2021   Prob (F-statistic):                    7.44e-12
Time:                        13:26:19   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

In [39]:
data = pd.DataFrame(X, columns=['col1','col2','col3'])
data['y'] = y

In [40]:
data

,col1,col2,col3,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941
...,...,...,...,...
95,0.502962,0.705644,-0.100857,0.253019
96,0.074699,-0.790788,0.603616,-0.148563
97,-0.473413,-1.094827,0.603872,0.348516
98,0.369967,1.004348,-0.172917,0.204080


In [41]:
result = smf.ols('y ~ col1 + col2 + col3', data).fit()

In [42]:
result.params

Intercept    0.033559
col1         0.176149
col2         0.224826
col3         0.514808
dtype: float64

In [43]:
result.tvalues

Intercept    0.952188
col1         3.319754
col2         4.850730
col3         6.303971
dtype: float64